In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
import urllib.request
import requests
import os 
import json
import sys 
import re
import time
import tqdm

###### Import selenium dependencies
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.keys import Keys

%matplotlib inline

#### Set up chrome options for the browser

In this section, we can add keyboard arguments to the chrome browser as options. The website https://peter.sh/experiments/chromium-command-line-switches/ contains all the possible extensions to the chrome option class. 

In [4]:
## Set up chrome Options 

chrome_options = Options()  
chrome_options.add_argument("--headless") 

#### Spider Class

In [5]:
class hoaxSpider():
    
    def __init__(self):
        
        self.URL="http://www.warriorelihoax.com"
        self.content = self.get_request()
        self.soup = self.get_soup()
        self.cases = self.get_names_cases()
        
    def get_request(self):
        content = requests.get(self.URL).content
        return content
    
    def get_soup(self):
        soup = BeautifulSoup(self.content,'html.parser')
        return soup
    
    def get_names_cases(self):
        all_tables = self.soup.find_all(id = "cat")
        txt = ""
        for i in all_tables:
            txt+=i.text.strip()
        cases = []
        for i in txt.split("\n"):
            reg = re.findall(r"^[^\(]+", i)
            cases.append(reg)
        flat_list = [item for sublist in cases for item in sublist]
        newL = []
        for i in flat_list:
            newL.append(i.replace(u'\xa0', u'').replace(" ", "-"))
        return newL[1:] 
    
    def select_hoax_case(self, dropdown = '//*[@id="cat"]'):
        
        self.driver = webdriver.Chrome()
        self.driver.get("http://www.warriorelihoax.com")
        python_button = self.driver.find_element_by_xpath(dropdown + '/option[5]')
        python_button.click()

    def scrape_images(self, dropdown = '//*[@id="cat"]'):
        
        #init a chrome driver. Add headless option!
        self.driver = webdriver.Chrome()
        self.driver.get(self.URL)
        
        # option is the number of cases. This variable is embedded into python_button in order to automatically 
        # change the pages
        option = 42
        while option:
            time.sleep(1)
            try:
                python_button = self.driver.find_element_by_xpath(dropdown + '/option[' + str(option) + ']')
         
                #get the link for every page. This involves to call the get_names_cases. 
                new_URL = self.URL + "/category/" + self.cases[option-1] + "/"
                python_button.click()
                
                #scrape for the images!
                self.URL = new_URL
                self.content = self.get_request()
                self.soup = self.get_soup()
                
                find_images = self.soup.find_all("a", href = True) #{"id": re.compile(r'\d+')}
                
                #create the directories
                dir_name = self.cases[option-1]
                if not os.path.exists(dir_name):
                    os.makedirs(dir_name)
                    if not os.path.exists(dir_name + "/Images"):
                        os.makedirs(dir_name + "/Images")
                        
                #download images and store to directory
                counter = 1
                for img in tqdm.tqdm(find_images):
                    try:
                        urllib.request.urlretrieve(img["href"], dir_name + "/Images/" + "Img_" + str(counter) + ".jpg")
                        counter += 1
                    except:
                        pass
                    
            except:
                break
            

            
            option += 1
        
        return find_images
            
            
            

In [6]:
spider = hoaxSpider()

In [7]:
case_names = spider.get_names_cases()

In [8]:
x = spider.scrape_images()

  0%|                                                                                          | 0/166 [00:00<?, ?it/s]

#content
Warrior-Eli-Hoax/Images/Img_1.jpg
http://www.warriorelihoax.com/
Warrior-Eli-Hoax/Images/Img_1.jpg


  1%|▉                                                                                 | 2/166 [00:00<00:31,  5.28it/s]

http://warriorelihoax.com/
Warrior-Eli-Hoax/Images/Img_2.jpg


  2%|█▍                                                                                | 3/166 [00:02<02:26,  1.11it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████ | 164/166 [01:14<00:01,  1.34it/s]

http://www.warriorelihoax.com/
Warrior-Eli-Hoax/Images/Img_140.jpg


 99%|███████████████████████████████████████████████████████████████████████████████▌| 165/166 [01:14<00:00,  1.60it/s]

https://wordpress.org/
Warrior-Eli-Hoax/Images/Img_141.jpg


100%|████████████████████████████████████████████████████████████████████████████████| 166/166 [01:15<00:00,  1.70it/s]


In [9]:
for img in x:
    print(img['href'])

#content
http://www.warriorelihoax.com/
http://warriorelihoax.com/
http://www.warriorelihoax.com/links/
http://www.warriorelihoax.com/report-a-suspected-hoax/
https://www.facebook.com/WarriorEliHoax
http://warriorelihoax.tumblr.com/
https://twitter.com/WarriorEliHoax
http://www.warriorelihoax.com/2013/01/10/emily-dirr-and-warrior-eli/
http://warriorelihoax.wordpress.com/2012/06/05/emilys-runaway-imagination/
http://warriorelihoax.wordpress.com/category/emily-dirr/
https://www.facebook.com/WarriorEliHoax
http://0370f4a.netsolhost.com/wordpress2/wp-content/uploads/2012/06/547930_10150950252901153_526411152_12003746_568653530_n-1.jpg
http://warriorelihoax.wordpress.com/2012/05/14/the-death-of-dana-dirr-16/
http://webcache.googleusercontent.com/search?q=cache:fyTqC-WSBMEJ:www.facebook.com/warrioreli/timeline+&cd=13&hl=en&ct=clnk&gl=us
http://0370f4a.netsolhost.com/wordpress2/wp-content/uploads/2012/06/6a00d8341c792353ef0168eb7c94d6970c-800wi1.jpg
http://mwopblog.com
http://www.tineye.com/
